In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import pandas as pd
import numpy as np  

import sys
sys.path.append('/Users/jessbreda/Desktop/github/jbreda_animal_training/notebooks/_active_experiments/FixationGrower')
# Path to the 'meetings' directory where your .py file resides
meetings_path = "/Users/jessbreda/Desktop/github/jbreda_animal_training/notebooks/meetings"
if meetings_path not in sys.path:
    sys.path.append(meetings_path)
import config_alpha_experiment as alpha_config

import behav_viz.visualize as viz
import behav_viz.utils as utils
from behav_viz.utils import plot_utils as pu
from TCM_mtg_2024_09_05_FixationExperiment import *
import behav_viz.ingest as ingest

FIGURES_PATH = utils.dir_utils.get_figures_path("rats", "alpha")

# settings
%load_ext autoreload
%autoreload 2
sns.set_context("talk")

DATA_PATH = "/Users/jessbreda/Desktop/github/jbreda_animal_training/data/meetings/2024_12_LabMeeting/"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0. Load Data


In [11]:
date_min = alpha_config.ALPHA_START_DATES["alpha_1"]
date_max = "2024-10-17"  # R056 was the last to finish on 2024-10-15
tdf = ingest.create_trials_df.create_trials_df_from_dj(
    alpha_config.ALPHA_ANIMALS, date_min, date_max=date_max
)

tdf = ingest.drop_dates(tdf, alpha_config.ALPHA_DATE_DROPS)

r044_max_date = pd.Timestamp(
    "2024-08-18"
).date()  # this animal had a rig break and needs to be dropped
tdf = tdf.query("not (animal_id == 'R044' and date > @r044_max_date)")

# remove post probe stages
tdf = tdf.query("stage > 4 and stage < 11")

animal_id R040
Fetched 75 dates from Sessions table from 2024-07-20 to 2024-10-17 for R040

animal_id R041
Fetched 84 dates from Sessions table from 2024-07-20 to 2024-10-17 for R041

animal_id R042
Fetched 84 dates from Sessions table from 2024-07-20 to 2024-10-17 for R042

animal_id R043
Fetched 85 dates from Sessions table from 2024-07-20 to 2024-10-17 for R043

animal_id R044
Fetched 50 dates from Sessions table from 2024-07-20 to 2024-09-21 for R044

animal_id R045
Fetched 83 dates from Sessions table from 2024-07-20 to 2024-10-17 for R045

animal_id R046
Fetched 84 dates from Sessions table from 2024-07-20 to 2024-10-17 for R046

animal_id R047
Fetched 78 dates from Sessions table from 2024-07-20 to 2024-10-17 for R047

animal_id R048
Fetched 74 dates from Sessions table from 2024-07-27 to 2024-10-17 for R048

animal_id R049
Fetched 69 dates from Sessions table from 2024-07-29 to 2024-10-17 for R049

animal_id R050
Fetched 75 dates from Sessions table from 2024-07-27 to 2024-10-1

In [12]:
days_df = ingest.create_days_df.create_days_df_from_dj(
    animal_ids=alpha_config.ALPHA_ANIMALS,
    date_min=date_min,
    date_max=date_max,
    verbose=True,
)

Fetched data from SessionAggDate table from 2024-07-20 to 2024-10-17  
Today is not being queried or data already exists in the database.
No new data from today, 2025-02-06 to add to SessionAggDate.

1365 daily summaries fetched for animals: 
['R040', 'R041', 'R042', 'R043', 'R044', 'R045', 'R046', 'R047', 'R048', 'R049', 'R050', 'R051', 'R052', 'R053', 'R054', 'R055', 'R056', 'R057']
between 2024-07-20 and 2024-10-17


## 1. Weight


In [14]:
days_df.groupby("animal_id")["mass"].describe()

,count,mean,std,min,25%,50%,75%,max
animal_id,,,,,,,,
R040,82.0,505.524390,29.897448,459.0,480.00,499.5,535.00,564.0
R041,84.0,497.202381,23.572385,458.0,475.75,496.5,520.00,534.0
R042,84.0,552.202381,27.156743,500.0,527.75,554.5,576.00,596.0
R043,84.0,489.297619,20.382948,455.0,469.00,486.5,510.00,522.0
R044,58.0,391.844828,12.429170,376.0,382.00,386.0,404.25,415.0
R045,83.0,514.481928,19.566872,480.0,500.00,515.0,529.50,550.0
R046,84.0,442.821429,26.023234,399.0,416.75,443.0,468.50,483.0
R047,82.0,461.365854,29.588333,420.0,430.00,462.0,490.75,501.0
R048,74.0,412.256757,12.050193,391.0,401.00,417.0,422.75,433.0


## 2. Antibias


In [17]:
tdf.query("ab_beta > 0").groupby("animal_id")["ab_beta"].describe()

,count,mean,std,min,25%,50%,75%,max
animal_id,,,,,,,,
R040,8287.0,2.879269,0.472111,0.1,3.0,3.0,3.0,3.0
R041,3939.0,2.765296,0.639377,0.1,3.0,3.0,3.0,3.0
R042,8957.0,2.893156,0.445795,0.1,3.0,3.0,3.0,3.0
R043,7274.0,2.887627,0.457969,0.1,3.0,3.0,3.0,3.0
R044,922.0,2.787419,0.676128,0.1,3.0,3.0,3.0,3.0
R045,6856.0,2.860414,0.505046,0.1,3.0,3.0,3.0,3.0
R046,6864.0,2.854240,0.515223,0.1,3.0,3.0,3.0,3.0
R047,10043.0,2.861107,0.504142,0.1,3.0,3.0,3.0,3.0
R048,17400.0,3.687802,0.928167,0.1,3.0,4.5,4.5,4.5


## 3. Water


In [15]:
tdf.query("water_delivered > 0").groupby("animal_id")["water_delivered"].describe()

,count,mean,std,min,25%,50%,75%,max
animal_id,,,,,,,,
R040,6561.0,30.000000,0.000000,30.0,30.0,30.0,30.0,30.0
R041,3409.0,30.000000,0.000000,30.0,30.0,30.0,30.0,30.0
R042,7109.0,30.000000,0.000000,30.0,30.0,30.0,30.0,30.0
R043,6793.0,30.000000,0.000000,30.0,30.0,30.0,30.0,30.0
R044,655.0,30.000000,0.000000,30.0,30.0,30.0,30.0,30.0
R045,6217.0,30.000000,0.000000,30.0,30.0,30.0,30.0,30.0
R046,5495.0,30.000000,0.000000,30.0,30.0,30.0,30.0,30.0
R047,9640.0,30.000000,0.000000,30.0,30.0,30.0,30.0,30.0
R048,11225.0,33.389131,4.360925,30.0,30.0,30.0,39.0,39.0
